(max_box:title)=
# Box with the Maximum Volume

Here, we attempt to optimize the shape of a box, in terms of its width \\(w\\), height \\(h\\), and depth \\(d\\), so that its volume is maximized, subject to a number of constraints. The model can be written as

$$
\begin{align}
\max~& whd \\
\text{s.t.}~&2(wh + dh) \leq A_{\text{wall}} \\
&wd \leq A_{\text{floor}} \\
&\alpha \leq h/w \leq \beta, \gamma \leq d/w \leq \delta, 
\end{align}
$$

where model parameters:
- Limits on wall area $A_{\text{wall}} = 200$,
- Limits on floor area $A_{\text{floor}} = 150$,
- Limits on height-width ratio: $\alpha=0.8$, and $\beta=1.5$,
- Limits on depth-width ratio: $\gamma=0.8$, and $\delta=1.5$.

The geometric program above is reformulated as decision variables are replaced by their logarithm transformations: $x = \log(w)$, $y = \log(h)$, and $z = \log(d)$, then we have

$$
\begin{align}
\max~& x + y + z \\
\text{s.t.}~&2(\exp(x + y) + \exp(z + y)) \leq A_{\text{wall}} \\
&\exp(x + z) \leq A_{\text{floor}} \\
&\exp(x - y) \leq \alpha, \exp(y - x) \leq \beta \\
&\exp(x - z) \leq \gamma, \exp(z - x) \leq \delta. \\
\end{align}
$$

Such an exponential cone program can be implemented by the code below.

In [13]:
from rsome import ro
from rsome import msk_solver as msk
import rsome as rso
import numpy as np

A_wall = 200
A_floor = 150
alpha, beta = 0.8, 1.5
gamma, delta = 0.8, 1.5

model = ro.Model()
x = model.dvar()
y = model.dvar()
z = model.dvar()
a = model.dvar()
b = model.dvar()

model.max(x + y + z)
model.st(rso.exp(x + y) <= a)
model.st(rso.exp(z + y) <= b)
model.st(2 * (a+b) <= A_wall)
model.st(rso.exp(x + z) <= A_floor)
model.st(rso.exp(x - y) <= alpha, rso.exp(y - x) <= beta)
model.st(rso.exp(x - z) <= gamma, rso.exp(z - x) <= delta)
model.solve(msk)

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0055s


Please note that RSOME does not support the summation of two or more exponential functions, such as the exponential function `rso.exp()`. This is why we introduced intermediate variables `a` and `b` to represent the epigraph of `rso.exp(x + y)` and `rso.exp(z + y)`, so that their summation can be formulated in constraints.

The optimal solution is presented in the following code segment.

In [14]:
print(x.get(), y.get(), z.get())

1.732868012650267 1.95601176128049 2.138332408802994


In [15]:
print(f'Width:  {np.exp(x.get()):.4f}')
print(f'Height: {np.exp(y.get()):.4f}')
print(f'Depth:  {np.exp(z.get()):.4f}')

Width:  5.6569
Height: 7.0711
Depth:  8.4853


Hence the optimal width, height, and depth are 5.6569, 7.0711, and 8.4853, respectively. 

In this example, the exponential cone programming problem is solved by the ECOS solver. If no exponential cone solver is available, the model can be approximated by a second-order cone program, using the approach mentioned in {cite:ps}`Zhu_Xie_Sim2022joint`. The following code solves the approximated model using the second-order cone solver Gurobi.

In [7]:
from rsome import grb_solver as grb

model.soc_solve(grb)

Being solved by Gurobi...
Solution status: 2
Running time: 0.0117s


The approximated solution is very close to the exact solution.

In [5]:
print(x.get(), y.get(), z.get())

1.7329191523155112 1.9560630196440272 2.138378987299716


In [6]:
print(f'Width:  {np.exp(x.get()):.4f}')
print(f'Height: {np.exp(y.get()):.4f}')
print(f'Depth:  {np.exp(z.get()):.4f}')

Width:  5.6571
Height: 7.0714
Depth:  8.4857


<br>

---

<font size="5">Reference</font>

```{bibliography}
:filter: docname in docnames
```